In [22]:
import swat
import os
from sasctl import Session
from sasctl.tasks import register_model, publish_model
from sasctl.services import model_repository

In [23]:
# Connect to the CAS server
os.environ['CAS_CLIENT_SSL_CA_LIST'] = '/opt/sas/viya/config/etc/SASSecurityCertificateFramework/cacerts/trustedcerts.pem'
s = swat.CAS('frasepviya35smp', 5570)

In [24]:
# Define default session caslib
s.sessionprop.setsessopt(caslib='Public')

# Name the CAS table
indata = 'HMEQ_TRAIN'

# Declare python CASTable view on CAS in-memory table
castbl = s.CASTable(indata)

NOTE: 'Public' is now the active caslib.


In [25]:
castbl.describe()

,BAD,LOAN,MORTDUE,VALUE,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
count,2381.000000,2381.000000,2178.000000,2339.000000,2184.000000,2105.000000,2153.000000,2255.000000,2189.000000,2290.000000,1873.000000
mean,0.199916,18216.463671,73888.350849,102594.821005,8.851671,0.230404,0.449141,178.606762,1.172225,21.217467,33.473330
std,0.400021,10669.354029,43967.539209,58102.560140,7.520705,0.754975,1.094231,85.735499,1.721473,10.055845,8.228359
min,0.000000,1300.000000,2619.000000,9100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.524499
25%,0.000000,10800.000000,46057.000000,66410.000000,3.000000,0.000000,0.000000,115.136885,0.000000,15.000000,28.638063
50%,0.000000,16000.000000,64811.000000,89079.000000,7.000000,0.000000,0.000000,172.696685,1.000000,20.000000,34.644922
75%,0.000000,23000.000000,92487.000000,121382.000000,13.000000,0.000000,0.000000,229.177959,2.000000,26.000000,38.942760
max,1.000000,89000.000000,369874.000000,855909.000000,41.000000,10.000000,10.000000,1168.233561,14.000000,71.000000,143.949605


# Train a gradient boosting model to identify iris species.

In [26]:
s.loadactionset('decisionTree')
castbl.decisiontree.gbtreetrain(target='BAD',
                             inputs=['BAD','LOAN','MORTDUE','VALUE','YOJ','DEROG','DELINQ','CLAGE','NINQ','CLNO','DEBTINC'],
                             savestate='gradboost_astore')

# Establish a reference to the newly created ASTORE table.
astore = s.CASTable('gradboost_astore')

NOTE: Added action set 'decisionTree'.
NOTE: 916779 bytes were written to the table "gradboost_astore" in the caslib "Public".


# Register the model in the central repository for versioning and deployment

In [27]:
with Session('frasepviya35smp', 'viyademo01', 'demopw'):
    # Register the trained model by providing:
    #  - the ASTORE containing the model
    #  - a name for the model
    #  - a name for the project
    #
    # NOTE: the force=True option will create the project if it does not exist.
    model = register_model(astore, 'Gradient Boosting Astore SWAT', 'HMEQ_Modeling', force=True)

NOTE: Added action set 'astore'.
NOTE: Cloud Analytic Services saved the file _8158653793EB4429BA7DCAC28.sashdat in caslib ModelStore.


In [37]:
with Session('frasepviya35smp', 'viyademo01', 'demopw'):
    model_repository.update_model(dict(function='Classification'))

ValueError: Could not find ETag for update of {'function': 'Classification'}.

# Optional : trigger model deployement and scoring from Python API

In [12]:
with Session('frasepviya35smp', 'viyademo01', 'demopw'):
    module = publish_model(model, 'maslocal')

    # sasctl wraps the published module with Python methods corresponding to the various steps defined in the module (like "predict").
    response = module.score(SepalLength=5.1, SepalWidth=3.5, PetalLength=1.4, PetalWidth=0.2)

NOTE: Added action set 'decisionTree'.
NOTE: 918707 bytes were written to the table "gradboost_astore" in the caslib "Public".
NOTE: Added action set 'astore'.
NOTE: Cloud Analytic Services saved the file _C2ADBCAAC4C2425389E5B4C23.sashdat in caslib ModelStore.


In [13]:
s.close()